In [1]:
import os
import random
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
def grayscale_conversion():
    folders = ["n02102318-cocker_spaniel", "n02105056-groenendael", "n02106382-Bouvier_des_Flandres",
               "n02107574-Greater_Swiss_Mountain_dog"]
    
    no_of_images = []
    for folder in folders:
        files = [f for f in os.listdir(f"../Cropped/{folder}") if os.path.isfile(os.path.join(f"../Cropped/{folder}", f))]
        no_of_images.append(len(files))
    
    collect = {}
    for folder, count in zip(folders, no_of_images):
        all_files = os.listdir(f"../Cropped/{folder}")
        collect[folder] = all_files
        
    
    grayscaled_images = {}
    for folder, images in collect.items():
        combine = []
        for img in images:
            path = f"../Cropped/{folder}/{img}"
            loaded_image = cv.imread(path)
            combine.append(cv.cvtColor(loaded_image, cv.COLOR_BGR2GRAY))
        grayscaled_images[folder] = combine
    
    return grayscaled_images 

In [3]:
def standardize_dataset():
    grayscaled_images = grayscale_conversion()
        
    standardized_images = {}
    for folder, images in grayscaled_images.items():
        combine = []
        for img in images:
            assert img is not None
            histogram, bins = np.histogram(img.flatten(), 256, [0,256])
            histogram = histogram / histogram.sum().ravel()
            combine.append(histogram)
        standardized_images[folder] = combine
    
    return standardized_images

In [4]:
def split_dataset():
    standardized_images = standardize_dataset()
    
    training_set = {}
    test_set = {}
    for folder in standardized_images:
        perce_80 = int(len(standardized_images[folder]) * 0.8)
        training_set[folder] = random.sample(standardized_images[folder], perce_80)
    
    for folder, images in standardized_images.items():
        combine = []
        for img in images:
            if not np.all(np.equal(img, training_set[folder]), axis=1).any():
                combine.append(img)             
        test_set[folder] = combine
    
    return training_set, test_set

In [5]:
def summ(train, test):
    count = 0
    for tr, te in enumerate(train, test):
        if tr == te:
            count += 1
    return count

def naive_bayes_classifier():
    training_set, test_set = split_dataset()
    gnb = GaussianNB()
    for folder, images in training_set.items():
        X_train, y_train = images        
        y_pred = gnb.fit(X_train, y_train)
#         print(f"Number of mislabeled points out of a total {len(images)} points: {summ(images, test_set[folder])}")

# naive_bayes_classifier()

In [8]:
# def AdaBoostClassifier():
training_set, testing_set = split_dataset()
training_data = []
testing_data = []
for train, test in zip(training_set, testing_set):
    training_data.extend(training_set[train])
    testing_data.extend(testing_set[test])

training_data = np.array(training_data)
testing_data = np.array(testing_data)
testing_data = testing_data.reshape(-1,1)
print("yes")

# AdaBoostClassifier()

yes


In [9]:
clf = AdaBoostClassifier()
clf.fit(training_data, testing_data)
print(clf.predict([[0, 0, 0, 0]]))
print(clf.score(training_data, testing_data))

C:\Users\noill\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Found input variables with inconsistent numbers of samples: [519, 33792]